The best dataset I've found so far contains is tatoeba.

https://tatoeba.org/fr/downloads

Another is conversational sentences from film and series subtitles with translations for multiple languages:

https://opus.nlpl.eu/OpenSubtitles-v2018.php
https://github.com/PolyAI-LDN/conversational-datasets

In [20]:
import os
import constants
import pandas as pd

In [21]:
constants.language_code = 'de'

In [22]:
#filepath_en = f"../input_files/{constants.language_code}/open_subtitles/OpenSubtitles_en-{constants.language_code}.en"
#filepath_lang = f"../input_files/{constants.language_code}/open_subtitles/OpenSubtitles_en-{constants.language_code}.{constants.language_code}"

filepath = f"../input_files/{constants.language_code}/tatoeba_{constants.language_code}_en.tsv"

df = pd.read_csv(filepath, sep='\\t')
#en_series = pd.read_csv(filepath_en, sep='\\t')
#lang_series = pd.read_csv(filepath_lang, sep='\\t')

C:\Users\Toby Usher\AppData\Local\Temp\ipykernel_14444\63767765.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filepath, sep='\\t')


In [23]:
df.head()

,77,Lass uns etwas versuchen!,1276,Let's try something.
0,80,Was ist das?,42849,What is it?
1,80,Was ist das?,66989,What's that?
2,80,Was ist das?,392210,What is this?
3,80,Was ist das?,413821,What's this?
4,80,Was ist das?,464592,What is that?


In [24]:
df.count

<bound method DataFrame.count of               77                          Lass uns etwas versuchen!      1276  \
0             80                                       Was ist das?     42849   
1             80                                       Was ist das?     66989   
2             80                                       Was ist das?    392210   
3             80                                       Was ist das?    413821   
4             80                                       Was ist das?    464592   
...          ...                                                ...       ...   
482459  12434546                           Tom ist nicht zu trauen.   3405189   
482460  12434546                           Tom ist nicht zu trauen.   3433971   
482461  12434546                           Tom ist nicht zu trauen.   1026813   
482462  12434547                         Tom kann man nicht trauen.   1026813   
482463  12434595  Marcus fragte sich zum hundertsten Mal, was de...  1243459

In [25]:
# Only keep the sentence and translation columns
df = pd.DataFrame({
    'sentence': df.iloc[:, 1],
    'translated_sentence': df.iloc[:, 3]
})

In [26]:
df.head()

,sentence,translated_sentence
0,Was ist das?,What is it?
1,Was ist das?,What's that?
2,Was ist das?,What is this?
3,Was ist das?,What's this?
4,Was ist das?,What is that?


In [27]:
"""# Combine into a dataframe and randomly sample n rows
n = 3_000_000

lang_series = lang_series.reset_index(drop=True)
en_series = en_series.reset_index(drop=True)

lang_series_list = lang_series.values.tolist()
en_series_list = en_series.values.tolist()

# Flatten the lists
lang_series_list = [item[0] for item in lang_series_list]
en_series_list = [item[0] for item in en_series_list]


df = pd.DataFrame({
    'sentence': lang_series_list,
    'translation': en_series_list
})

# First 100 or so lines are in English for some reason
#df = df[df.index > 100]

df_sample = df.sample(n, random_state=1)
df_sample.to_csv(f'../input_files/{constants.language_code}/open_subtitles_uncleaned_sentences.csv', sep='\t')
"""

"# Combine into a dataframe and randomly sample n rows\nn = 3_000_000\n\nlang_series = lang_series.reset_index(drop=True)\nen_series = en_series.reset_index(drop=True)\n\nlang_series_list = lang_series.values.tolist()\nen_series_list = en_series.values.tolist()\n\n# Flatten the lists\nlang_series_list = [item[0] for item in lang_series_list]\nen_series_list = [item[0] for item in en_series_list]\n\n\ndf = pd.DataFrame({\n    'sentence': lang_series_list,\n    'translation': en_series_list\n})\n\n# First 100 or so lines are in English for some reason\n#df = df[df.index > 100]\n\ndf_sample = df.sample(n, random_state=1)\ndf_sample.to_csv(f'../input_files/{constants.language_code}/open_subtitles_uncleaned_sentences.csv', sep='\t')\n"

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482464 entries, 0 to 482463
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sentence             482464 non-null  object
 1   translated_sentence  482464 non-null  object
dtypes: object(2)
memory usage: 7.4+ MB


In [29]:
# Let's see if there are any duplicates in the dataset
df[df["sentence"].duplicated(keep=False)].sort_values("sentence").count

<bound method DataFrame.count of                                                  sentence  \
62976   "Arbeit macht frei" ist ein Spruch, der in der...   
62977   "Arbeit macht frei" ist ein Spruch, der in der...   
333291       "Gibt's was Neues?" "Nein, alles wie immer."   
333292       "Gibt's was Neues?" "Nein, alles wie immer."   
393807                    "Hallo!'' ''Hallo, wie geht's?"   
...                                                   ...   
391507                „Wo wohntest du?“ – „In Amsterdam.“   
417655  „Wollen wir bis morgen warten?“ – „Gut, aber n...   
417656  „Wollen wir bis morgen warten?“ – „Gut, aber n...   
332631  „Worauf hast du heute Lust? Auf italienisch, g...   
332630  „Worauf hast du heute Lust? Auf italienisch, g...   

                                      translated_sentence  
62976   "Arbeit macht frei", meaning "Work sets you fr...  
62977   "Arbeit macht frei", meaning "Work sets you fr...  
333291                   "Any news?" "No, same as usua

In [30]:
df.dtypes

sentence               object
translated_sentence    object
dtype: object

In [31]:
# Remove all duplicates from the dataframe
df = df.drop_duplicates("sentence")

In [32]:
df.info

<bound method DataFrame.info of                                                  sentence  \
0                                            Was ist das?   
5       Heute ist der 18. Juni und das ist der Geburts...   
6         Herzlichen Glückwunsch zum Geburtstag, Muiriel!   
7                                   Muiriel ist jetzt 20.   
9                             Das Passwort ist "Muiriel".   
...                                                   ...   
482456                       Halte uns auf dem Laufenden.   
482457                    Das ist keine gute Übersetzung.   
482459                           Tom ist nicht zu trauen.   
482462                         Tom kann man nicht trauen.   
482463  Marcus fragte sich zum hundertsten Mal, was de...   

                                      translated_sentence  
0                                             What is it?  
5        Today is June 18th and it is Muiriel's birthday!  
6                                Happy birthday, Muirie

In [33]:
lengths: pd.Series = df['sentence'].str.len()
max_characters: int = lengths.max()
max_index = lengths.idxmax()

# Find the sentence with the most characters to see if there are any delimitation issues.
print(f'Longest sentence: {max_characters} characters')
print(df[df.index == max_index]['sentence'].values[0][:600]) # Print out first 600 characters

Longest sentence: 1427 characters
In diesem Stadium der Weltgeschichte gibt es so etwas wie eine unabhängige Presse in den Vereinigten Staaten von Amerika nicht. Sie wissen es und ich weiß es. Unter Ihnen ist nicht Einer, der sich traut, seine ehrliche Meinung zu schreiben, und täten Sie es, wüssten Sie von vornherein, dass es niemals gedruckt werden würde. Ich werde wöchentlich dafür bezahlt, dass ich meine ehrlichen Ansichten draußenlasse aus der Zeitung, mit der ich in Verbindung stehe. Andere unter Ihnen bekommen vergleichbare Gehälter bezahlt für vergleichbare Dinge, und jeder von Ihnen, der so töricht wäre, ehrliche Ansi


In [34]:
# Cut off any sentences longer than 200 or 
# shorter than 30 characters
df = df[
    (df['sentence'].str.len() < 200)
    & (df['sentence'].str.len() > (15 if constants.language_code == 'th' else 30)) # setting to 30 cuts out most thai sentences
     ]


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271818 entries, 5 to 482463
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sentence             271818 non-null  object
 1   translated_sentence  271818 non-null  object
dtypes: object(2)
memory usage: 6.2+ MB


In [36]:
# Randomly sample n rows to get a reduced dataset for easier training while testing out this method. Set a seed for reproducability.
#n_rows = 30000

#reduced_df = df.sample(n=n_rows, random_state=1)

In [37]:
# Remove id column and save dataframes as csv
df.to_csv(f"../output_files/{constants.language_code}/step0_sentences.csv", sep='\t', index=False)
#reduced_df.to_csv("./french_sentences_reduced.csv", sep='\t', index=False)